In [ ]:
# Cell 1: Import Libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import kruskal

# --- Initial Setup for Plots ---
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 7)

In [ ]:
# Cell 2: Load and Combine Cleaned Data
# This list directly corresponds to the consistent filenames in your 'data/' folder.
countries = ['benin', 'sierra_leone', 'togo']
df_list = []

print("--- Loading Cleaned Datasets ---")
for country in countries:
    file_path = f'../data/{country}_clean.csv'
    try:
        # Load the CSV and ensure the 'Timestamp' column is parsed as dates
        df = pd.read_csv(file_path, parse_dates=['Timestamp'])
        # Add a 'country' column with a nicely formatted name (e.g., 'Sierra Leone')
        df['country'] = country.replace('_', ' ').title()
        df_list.append(df)
        print(f"Successfully loaded: {file_path}")
    except FileNotFoundError:
        print(f"ERROR: File not found at '{file_path}'. Please ensure the file exists and the name is correct.")

# Concatenate all individual DataFrames into a single one
if df_list:
    df_all = pd.concat(df_list, ignore_index=True)
    print("\nAll cleaned data loaded and combined successfully.")
    print("Combined DataFrame Shape:", df_all.shape)
    print("Data types:\n", df_all.dtypes)
    print("\nFirst 5 rows of combined data:")
    print(df_all.head())
else:
    print("\nNo data was loaded. Halting execution.")
    # In a real notebook, you might stop here or handle the error gracefully.
    df_all = pd.DataFrame() # Create empty df to avoid errors in subsequent cells

In [ ]:
# Cell 3: Metric Comparison - Boxplots
if not df_all.empty:
    print("\n--- Generating Metric Comparison Boxplots ---")
    # Define the key metrics to compare
    metrics = ['GHI', 'DNI', 'DHI']

    # Create a figure with 3 subplots (one for each metric)
    fig, axes = plt.subplots(1, 3, figsize=(20, 7), sharey=True)
    fig.suptitle('Side-by-Side Comparison of Solar Irradiance Metrics', fontsize=16)

    for i, metric in enumerate(metrics):
        sns.boxplot(ax=axes[i], x='country', y=metric, data=df_all, palette='viridis')
        axes[i].set_title(f'{metric} Distribution by Country', fontsize=12)
        axes[i].set_xlabel('Country', fontsize=10)
        axes[i].set_ylabel(f'{metric} (W/m²)' if i == 0 else '', fontsize=10) # Label y-axis only on the first plot
        axes[i].tick_params(axis='x', rotation=15)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to make space for suptitle
    plt.show()

In [ ]:
# Cell 4: Summary Table
if not df_all.empty:
    print("\n--- Summary Statistics Across Countries ---")
    # Group by country and calculate mean, median, and std for the key metrics
    summary_table = df_all.groupby('country')[['GHI', 'DNI', 'DHI']].agg(['mean', 'median', 'std'])
    
    # Display the formatted table
    display(summary_table.style.format("{:.2f}").set_caption("Mean, Median, and Standard Deviation of Solar Metrics"))

In [ ]:
# Cell 5: Statistical Testing (Kruskal-Wallis)
if not df_all.empty:
    print("\n--- Statistical Significance Test for GHI ---")
    # Kruskal-Wallis is a good non-parametric choice if we don't assume normal distribution.

    # Prepare the data groups for the test
    ghi_benin = df_all[df_all['country'] == 'Benin']['GHI'].dropna()
    ghi_sl = df_all[df_all['country'] == 'Sierra Leone']['GHI'].dropna()
    ghi_togo = df_all[df_all['country'] == 'Togo']['GHI'].dropna()

    # Perform the test
    stat, p_value = kruskal(ghi_benin, ghi_sl, ghi_togo)
    
    print(f"Kruskal-Wallis H-test for GHI distributions:")
    print(f"P-value: {p_value:.4f}")

    # Interpret the result
    alpha = 0.05
    if p_value < alpha:
        print(f"Result: Since the p-value ({p_value:.4f}) is less than {alpha}, we reject the null hypothesis.")
        print("This indicates there is a statistically significant difference in the GHI distributions among the countries.")
    else:
        print(f"Result: Since the p-value ({p_value:.4f}) is greater than {alpha}, we fail to reject the null hypothesis.")
        print("This indicates there is no statistically significant difference in the GHI distributions among the countries.")

# Cell 6: Key Observations and Strategic Recommendation

### Key Observations from Cross-Country Comparison:

*   **Highest Solar Potential:** Based on the boxplots and summary table, **Togo** consistently shows the highest median Global Horizontal Irradiance (GHI). This suggests it receives the most overall solar energy, making it a prime candidate for solar farm investment.

*   **Greatest Variability:** **Sierra Leone** exhibits the largest interquartile range and standard deviation for GHI. This indicates greater variability and less predictable solar conditions, which could be due to more frequent cloud cover or changing weather patterns. While still viable, this could pose a risk to consistent energy generation and may require more advanced grid management or energy storage solutions.

*   **Direct vs. Diffuse Light:** While Togo leads in overall GHI, it's important to analyze the components. If a country has lower Direct Normal Irradiance (DNI) but higher Diffuse Horizontal Irradiance (DHI), it might influence the type of solar panel technology chosen. For example, certain panel types perform better in diffuse light conditions.

### Strategic Recommendation for MoonLight Energy Solutions:

Based on this initial analysis, the data strongly supports a recommendation to **prioritize Togo for the next phase of solar farm development.** Its high and consistent GHI suggests a more reliable and higher energy yield. A follow-up analysis should focus on identifying specific high-potential sites within Togo. Sierra Leone should be considered a secondary target, with the caveat that its energy output may be more variable, requiring investment in mitigation strategies like battery storage.

In [ ]:
# Cell 7: (Bonus) Visual Summary - Bar Chart
if not df_all.empty:
    print("\n--- (Bonus) Visual Summary: Average GHI Ranking ---")
    # Calculate and sort the mean GHI for each country
    avg_ghi = df_all.groupby('country')['GHI'].mean().sort_values(ascending=False)

    plt.figure(figsize=(8, 5))
    avg_ghi.plot(kind='bar', color=['#440154', '#21918c', '#fde725'], edgecolor='black') # Using viridis colors
    
    plt.title('Average GHI Ranking by Country', fontsize=14)
    plt.ylabel('Average GHI (W/m²)', fontsize=12)
    plt.xlabel('Country', fontsize=12)
    plt.xticks(rotation=0, fontsize=11)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()